# `iterator`-`generator`-`yield`

### 참고
* http://www.programiz.com/python-programming/iterator

## Sequence
- list, tuple, set, dictionary 등
- n개의 데이터를 가진 시퀀스는 그만큼의 메모리를 차지
- 이미 알고 있는 유한한 길이의 데이터 묶음에 대해서만 처리 가능

In [1]:
# 999,990,000개의 데이터를 모두 갖지 않음
r = range(10000, 1000000000)

In [2]:
# 필요시 값 계산
r[45000030]

45010030

## Iterator
* 일련의 데이터 값들을 가지고 있음
* `iterator`는 한번에 하나의 원소를 반환하는 객체
* `iterator` 객체는 두 개의 특별한 함수를 제공
  * `__iter__()` : iterator 자신을 반환
  * `__next__()`
* 어떤 객체가 **iterable**하다고 하는 것은, 그 객체로부터 `iterator`를 얻을 수 있다는 뜻
  * 예) list, tuple, string, ...
  * `iter()` 함수가 `iterator` 객체를 얻어 줌

In [3]:
# for문에서 사용 가능한 객체들은 iterable - list
for i in [1, 2, 3, 4]:
    print(i)

1
2
3
4


In [4]:
# for문에서 사용가능한 객체들은 iterable - dictionary
for k in {'x':1, 'y':2}:
    print(k)

x
y


In [5]:
# for문에서 사용가능한 객체들은 iterable - file
for line in open("iter.txt"):
    print(line,)

first line

second line



In [6]:
# iterable 객체는 join에서 사용가능
','.join(['a', 'b', 'c'])

'a,b,c'

In [7]:
# iterable 객체는 join에서 사용가능
','.join({'x':1, 'y':2})

'x,y'

In [8]:
# iterable 객체는 list로 변환 가능
list("python")

['p', 'y', 't', 'h', 'o', 'n']

In [9]:
# iterable 객체는 list로 변환 가능
list({'x':1, 'y':2})

['x', 'y']

### `iterator`를 이용한 loop
- iterable 객체들은 원하는만큼 읽고 처리하기 쉬워 간편한데, 모든 항목들이 필요하지 않더라도 많은 양을 메모리에 담고 있어야 하는 문제

In [10]:
my_list = [4, 7, 0, 3]
my_iter = iter(my_list)
my_iter

In [11]:
next(my_iter)

4

In [12]:
next(my_iter)

7

In [13]:
my_iter.__next__()

0

In [14]:
my_iter.__next__()

3

In [15]:
next(my_iter)    # 더이상 없으므로 오류

StopIteration: 

In [16]:
# for문에서 사용하는 iterator
for i in my_list:
    print(i)

4
7
0
3


#### `for`문의 실제 동작
* `for`문은 어떤 iterable이라도 받아들여 순환함

>```python
for element in iterable:
    # do something with element
```

* 이는 다음과 동일

>```python
>iter_obj = iter(iterable)   # iterable로부터 iterator 생성
>
># 무한 반복
>while True:
>    try:
>        element = next(iter_obj)  # 다음 원소 추출
>        # do something with element
>    except StopIteration:
>        break
>```

### 새로운 `iterator` 만들기
* `__iter__()`와 `__next__()` 메소드를 가진 클래스를 구현하면 됨
* `__iter__()`는 iterator 객체 자신을 반환하면 됨(필요에 따라 초기화)
* `__next__()`는 다음 원소를 반환하면 됨. 끝에 다다르면 **`StopIteration`** 예외 발생

In [17]:
# 2의 제곱으로 이루어진 iterator
class PowTwo:
    def __init__(self, max = 0):
        self.max = max
    def __iter__(self):
        self.n = 0
        return self
    def __next__(self):
        if self.n <= self.max:
            result = 2 ** self.n
            self.n += 1
            return result
        else:
            raise StopIteration

In [18]:
a = PowTwo(4)
i = iter(a)
next(i)

1

In [19]:
next(i)

2

In [20]:
next(i)

4

In [21]:
next(i)

8

In [22]:
next(i)

16

In [23]:
next(i)    # raise StopIteration

StopIteration: 

In [24]:
# for문에서 사용하는 iterator
for i in PowTwo(5):
    print(i)

1
2
4
8
16
32


### 무한 `iterator`
* `iterator`가 항상 끝이 있는 유한 길이일 필요 없음
* python의 `iter()`함수는 두 개의 인수를 받아들이는데, 첫번째는 호출가능한 함수, 두번째는 경계값(sentinel)
* `iterator`가 반환하는 값이 경계값(sentinel)과 같을 때까지 첫번째 인수의 함수를 호출

In [25]:
int()  # 항상 0을 반환하는 함수

0

In [26]:
inf = iter(int, 1)
next(inf)

0

In [27]:
next(inf)

0

In [28]:
# 자체적으로 만든 무한 iterator
# 홀수를 계속 반환
class InfIter:
    def __iter__(self):
        self.num = 1
        return self
    def __next__(self):
        num = self.num
        self.num += 2
        return num

a = iter(InfIter())

In [29]:
next(a)

1

In [30]:
next(a)

3

In [31]:
next(a)

5

In [32]:
next(a)

7

----
## Generator

* `iterator`에서 `__iter__()`와 `__next__()`를 이용하여 클래스 구현
* 이는 내부 상태를 계속 유지하면서, 반환할 값이 없으면 `StopIteration` 예외를 발생하도록 하는 부하가 많음

* **Generator**는 `iterator`를 만드는 손쉬운 방법이면서, 위의 부하를 자동으로 다루어 줌
* 단순하게 얘기하면, **generator는 (iterator) 객체를 반환하는 함수**

### generator 생성
* 단순하게는 `return` 대신, `yield`문을 사용하면 됨
  * `return`과 `yield` 모두 함수에서 어떤 값을 반환하는 구문
  * `return`은 함수를 완전히 끝내버리는 반면, **`yield`는 함수의 모든 상태를 저장하고 잠심 멈춘 후 나중에 호출되면 계속 이어서 수행**
* generator 함수와 일반 함수의 차이점
  - 하나 이상의 `yield`문이 포함되어 있음
  - generator함수가 호출되면 iterator 객체를 반환하지만, 즉시 실행을 시작하지 않음
  - `__iter__()`나 `__next__()`같은 메소드는 자동 구현됨
  - `yield`가 수행되면, **함수의 진행을 멈추고** 호출한 쪽으로 제어를 넘김
  - 지역 변수와 상태는 다음 호출될 때까지 기억됨
  - 함수가 종료될 때 자동으로 `StopIteration` 예외를 발생시킴

In [33]:
def my_gen():
    n = 1
    print("This is printed first")
    yield n

    n += 1
    print("This is printed second")
    yield n

    n += 1
    print("This is printed at last")
    yield n

In [34]:
a = my_gen()
next(a)

This is printed first


1

In [35]:
next(a)

This is printed second


2

In [36]:
next(a)

This is printed at last


3

In [37]:
next(a)    # raise StopIteration

StopIteration: 

In [38]:
# iterator처럼 generator도 for loop에서 사용 가능
for item in my_gen():
    print(item)

This is printed first
1
This is printed second
2
This is printed at last
3


In [39]:
# 유용한 generator 사용법
def revStr(my_str):
    length = len(my_str)
    for i in range(length-1, -1, -1):
        yield my_str[i]

for ch in revStr("hello"):
    print(ch, end='')

olleh

### Generator 표현식
* `lambda` 함수처럼 사용하는 `generator expression`
* list comprehesion과 유사하지만 **각괄호([ ])가 아닌 소괄호(( ))** 사용
* list comprehesion은 전체 리스트를 생성해 내는 반면, **generator expression은 한번에 하나의 원소만 생성**
  * 즉, 요청할 때마다 원소 하나 생성
  * 메모리 절약과 효율화 가능

In [40]:
my_list = [1, 3, 6, 10]

# list comprehesion
[x**2 for x in my_list]

[1, 9, 36, 100]

In [41]:
# generator expression
a = (x**2 for x in my_list)
a

<generator object <genexpr> at 0x7f6f6773a4c0>

In [42]:
next(a)

1

In [43]:
next(a)

9

In [44]:
next(a)

36

In [45]:
next(a)

100

In [46]:
next(a)    # raise StopIteration

StopIteration: 

In [47]:
# 함수의 인수로 사용
sum(x**2 for x in my_list)

146

In [48]:
max(x**2 for x in my_list)

100

### 왜 Generator방식을 사용하는가?
- 구현이 쉽고,
- 메모리 효율적이며,
- 무한 데이터 생성 가능
 >```python
 def all_even():
    n = 0
    while True:
        yield n
        n += 2
```

In [49]:
def only_odds(nums):
    for n in nums:
        # 홀수이면
        if n % 2 == 1:
            yield n

In [50]:
odds = only_odds(range(100))
# return처럼 반환하되, yield에서 함수의 진행이 멈춤
next(odds)

1

In [51]:
# next()로 다음 iteration으로 넘어가며, next()하는만큼만 함수 실행(lazy iteration)
next(odds)

3

In [52]:
def odd_square1(n):
    seq = range(n)
    seq = only_odds(seq)
    return  (x**2 for x in seq)

# 단지 generator만 생성
x = odd_square1(1000000)
x

<generator object odd_square1.<locals>.<genexpr> at 0x7f6f677affc0>

In [53]:
# 이제서야 iteration 수행
# generator를 사용하지 않으면 위 코드는 250만번 iteration
list(x)[:10]

[1, 9, 25, 49, 81, 121, 169, 225, 289, 361]

In [54]:
# odd_square1()과 같지만 반환값이 generator가 아닌 리스트
def odd_square2(n):
    seq = list(range(n))
    seq = list(only_odds(seq))
    return [x**2 for x in seq]

In [55]:
# generator 사용이 더 빠름
%timeit odd_square1(1000000)

1.35 µs ± 1.81 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [56]:
%timeit odd_square2(1000000)

387 ms ± 995 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### iterator vs. generator

In [57]:
# iterator 구현
class PowTwo:
    def __init__(self, max = 0):
        self.max = max

    def __iter__(self):
        self.n = 0
        return self

    def __next__(self):
        if self.n >= self.max:
            raise StopIteration

        result = 2 ** self.n
        self.n += 1
        return result

In [58]:
a = PowTwo(3)
i = iter(a)

In [59]:
list(i)

[1, 2, 4]

In [60]:
# generator 구현
def PowTwoGen(max = 0):
    n = 0
    while n < max:
        yield 2 ** n
        n += 1

In [61]:
x = PowTwoGen(3)
x

<generator object PowTwoGen at 0x7f6f677afbf8>

In [62]:
list(x)

[1, 2, 4]

### 무한 데이터

In [63]:
# 함수 호출할 때마다 다음 짝수를 무한 생성
def all_even():
    n = 0
    while True:
        yield n
        n += 2

In [64]:
x = all_even()
x

<generator object all_even at 0x7f6f6773a888>

In [65]:
next(x)

0

In [66]:
next(x)

2

### generator는 한번만 사용 가능

In [67]:
def gen():
    yield 1
    yield 2
    yield 3

In [68]:
x = gen()
x

<generator object gen at 0x7f6f6773a990>

In [69]:
list(x)

[1, 2, 3]

In [70]:
list(x)

[]

In [71]:
# T자형 파이프처럼
x = gen()

import itertools
x1, x2 = itertools.tee(x)

In [72]:
list(x1)

[1, 2, 3]

In [73]:
list(x2)

[1, 2, 3]

 * generator 연결(pipeline)
   - 예를 들어, 음식점 로그 파일 sells.log이 있다고 가정
   - 파일 안의 4번째 열에 매일 시간당 팔린 피자 개수를 나타내는 값이 있고,
   - 5년동안 팔린 전체 피자개수 합을 원한다고 하자.
   
>```python
with open('sells.log') as file:
    pizza_col = (line[3] for line in file)   # generator
    per_hour = (int(x) for x in pizza_col if x != 'N/A')  # generator
    print("Total pizzas sold = ", sum(per_hour))
```